# 1. Read and parse the training data

In [1]:
import pandas as pd

# Replace with your actual GitHub repo URL
url = "https://raw.githubusercontent.com/dgront/chem-ml/refs/heads/main/INPUTS/membrane_proteins/pdbtm.dat"

try:
  df = pd.read_csv(url, sep='\t')
  print(df.head())  # Display the first few rows of the DataFrame
except Exception as e:
  print(f"An error occurred: {e}")


  pdb_id chain_code                                           sequence  \
0   1a0b          _  TTEENSKSEALLDIPMLEQYLELVGPKLITDGLAVFEKMMPGYVSV...   
1   2a00          A  MGHHHHHHHHHHSSGHGGRHNRQASEFIPAQGVDEKTLADAAQLAS...   
2   7a0w          A  MKHLHRFFSSDASGGIILIIAAALAMLMANMGATSGWYHDFLETPV...   
3   7a0w          B  MKHLHRFFSSDASGGIILIIAAALAMLMANMGATSGWYHDFLETPV...   
4   2a06          A  TATYAQALQSVPETQVSQLDNGLRVASEQSSQPTCTVGVWIDAGSR...   

                                        region_types  
0  UUUUUU1111111111111111111111111111111111111111...  
1  UUUUUUUUUUUUUUUUUUUU11111111111111111111111111...  
2  UUUUUUUUUUU111HHHHHHHHHHHHHH222222222222222222...  
3  UUUUUUUUUU1111HHHHHHHHHHHHHH222222222222222222...  
4  2222222222222222222222222222222222222222222222...  


In [2]:
sequences = df['sequence'].tolist()
region_types = df['region_types'].tolist()
print(len(sequences), len(region_types))

65705 65705


# 2. Encode both inputs and outputs in one-hot encoding
write a function that takes 2 arguments:
  (1) the list of K characters allowed, say "12FHU.LBI"
  (2) the string to be encoded
The function should return a list of lists or a numpy array NxK with 0 and 1

# 3. Convert to tensors
Make list of "features" tensors Nx20, one per each input sequence; also "labels" tensors Nx9

# 4. Split data into training and validation sets

Use existing function as below

In [ ]:
from sklearn.model_selection import train_test_split
# split data into two sets: training and validation

features_train, features_val, labels_train, labels_val = train_test_split(features, labels, test_size=0.3, random_state=42)

# 5. Convert tensors into DataSets

In [ ]:
def create_dataset(features_list, labels_list):
    return tf.data.Dataset.from_generator(
        lambda: zip(features_list, labels_list),
        output_signature=(
            tf.TensorSpec(shape=(None, 20), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 9), dtype=tf.float32)
        )
    ).batch(1)

training_dataset = create_dataset(features_train, labels_train)
validation_dataset = create_dataset(features_val, labels_val)

# 6. Build the model (Neural Network)

In [5]:
# Use "None" because of variable-length sequences
import tensorflow as tf
input_layer = tf.keras.Input(shape=(None, 20))                                                  # First = input layer

x2 = tf.keras.layers.Conv1D(64, kernel_size=3, padding='same', activation='relu')(input_layer)  # Second layer = first hidden layer
x3 = tf.keras.layers.Conv1D(64, kernel_size=3, padding='same', activation='relu')(x2)           # Third layer = second hidden layer

output = tf.keras.layers.Conv1D(9, kernel_size=1, activation='softmax')(x3)                     # Fourth = output layer

model = tf.keras.Model(inputs=input_layer, outputs=output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # use sparse_categorical_crossentropy if labels are integers
    metrics=['categorical_accuracy']
)

# 7. Train the model

In [ ]:
hist = model.fit(training_dataset, validation_data=validation_dataset, epochs=100)